# 3.0 Pose Spatial Classifier

`PoseSpatialClassifier` class is designed to classify poses based on the added pose dimensions.

### Setup and Modules

In [22]:
import re
import pandas as pd

### Implementation

1. `get_face_position()`:  Determines if the face is oriented towards the `front` or `back` based on the z-axis value of the head..
2. `get_body_position()`: Classifies body position as `inverted`, `upright`, or `horizontal` by comparing the vertical alignment of head, chest, stomach, and hips.
3. `get_legs_position()`: Classifies legs position by scoring differences in mid-hip-to-knees, shoulder-to-knees, and hips-to-feet angles against reference legs positions.
4. `get_grip_position()`: Classifies grip position by scoring differences in shoulder-to-wrist, elbow-to-knuckles, and thumb-to-knuckles angles against reference grip positions.
5. `get_body_position_undefined()`, `get_legs_position_undefined()`, `get_grip_position_undefined()`: These functions handle cases where standard position classification criteria are not met, using a more tolerant approach to find the closest reference match or returning 'unknown' if no close match is found.

In [23]:
class PoseSpatialClassifier:
    def __init__(self, data, ref_legs, ref_grip):
        self.data = data
        self.ref_legs = ref_legs
        self.ref_grip = ref_grip

        self.body_match_count = 0
        self.body_undefined_count = 0

        self.legs_match_count = 0
        self.legs_undefined_count = 0

        self.grip_match_count = 0
        self.grip_undefined_count = 0

        self.data['pos_face'] = self.data.apply(self.get_body_position, axis=1)
        self.data['pos_body'] = self.data.apply(self.get_face_position, axis=1)
        self.data['pos_legs'] = self.data.apply(self.get_legs_position, axis=1)
        self.data['pos_grip'] = self.data.apply(self.get_grip_position, axis=1)

    def get_face_position(self, row):
        return "front" if row["head_z"] < 0 else "back"

    def get_body_position(self, row):
        self.body_match_count += 1
        if row['head_y'] > row['chest_y'] > row['stomach_y'] > row['hip_y']:
            return "inverted"
        elif row['head_y'] < row['chest_y'] < row['stomach_y'] < row['hip_y'] and abs(row['head_x'] - row['hip_x']) < 0.2:
            return "upright"
        elif row['head_y'] < row['hip_y'] and abs(row['head_y'] - row['hip_y']) < 0.2:
            return "horizontal"
        else:
            result = self.get_body_position_undefined(row)
            return result        

    def get_body_position_undefined(self,row):
        self.body_undefined_count += 1
        if row['head_y'] > row['stomach_y'] or row['stomach_y'] > row['hip_y']:
            return "inverted"
        if abs(row['head_y'] - row['hip_y']) < 0.2:
            return "horizontal"
        return "undefined"
    
    def get_legs_position(self, row):
        a_cols = ['a_mid_hip_to_knees', 'a_rgt_hip_to_foot', 'a_rgt_foot_to_ankle', 'a_lft_hip_to_foot', 'a_lft_foot_to_ankle']
        
        scores = {}
        for _, ref_row in self.ref_legs.iterrows():
            score = 0
            
            for col in a_cols:
                threshold = 10
                difference = abs(row[col] - ref_row[col])
                if difference <= threshold:
                    score += 1
            
            scores[ref_row['pose_name']] = score
        
        closest_match_spec = max(scores, key=scores.get)
        highest_score = scores[closest_match_spec]
        
        if highest_score == 0:
            return self.get_legs_position_undefined(row)
        else:
            self.legs_match_count += 1
            suffixes_to_remove = r'(-rgt|-lft|-inv|-rgt-inv|-lft-inv|-center|)$'
            closest_match = re.sub(suffixes_to_remove, '', closest_match_spec)
            return closest_match

    def get_legs_position_undefined(self, row):
        self.legs_undefined_count += 1
        a_cols = ['a_rgt_hip_to_foot', 'a_rgt_foot_to_ankle', 'a_lft_hip_to_foot', 'a_lft_foot_to_ankle']
        
        scores = {}
        for _, ref_row in self.ref_legs.iterrows():
            score = sum(abs(row[col] - ref_row[col]) for col in a_cols)
            scores[ref_row['pose_name']] = score
    
        closest_match_spec = min(scores, key=scores.get)
        closest_score = scores[closest_match_spec]
    
        if closest_score > 200:
            return 'unknown'
        else:
            suffixes_to_remove = r'(-rgt|-lft|-inv|-rgt-inv|-lft-inv|-center|)$'
            closest_match = re.sub(suffixes_to_remove, '', closest_match_spec)
            return closest_match
            
    def get_grip_position(self, row):
        a_cols = ['a_rgt_shoulder_to_wrist', 'a_rgt_elbow_to_knuckles','a_rgt_thumb_to_knuckles',
                  'a_lft_shoulder_to_wrist', 'a_lft_elbow_to_knuckles','a_lft_thumb_to_knuckles',]

        scores = {}
        for _, ref_row in self.ref_grip.iterrows():
            score = 0
            
            for col in a_cols:
                threshold = 10
                difference = abs(row[col] - ref_row[col])
                if difference <= threshold:
                    score += 1
            
            scores[ref_row['pose_name']] = score
        
        closest_match_spec = max(scores, key=scores.get)
        highest_score = scores[closest_match_spec]
        
        if highest_score == 0:
            return self.get_grip_position_undefined(row)
        else:
            self.grip_match_count += 1
            suffixes_to_remove = r'(-rgt|-lft|-inv|-rgt-inv|-lft-inv|-center|)$'
            closest_match = re.sub(suffixes_to_remove, '', closest_match_spec)
            return closest_match

    def get_grip_position_undefined(self, row):
        self.grip_undefined_count += 1
        a_cols = ['a_rgt_shoulder_to_wrist', 'a_rgt_elbow_to_knuckles','a_rgt_thumb_to_knuckles',
                  'a_lft_shoulder_to_wrist', 'a_lft_elbow_to_knuckles','a_lft_thumb_to_knuckles',]
        
        scores = {}
        for _, ref_row in self.ref_grip.iterrows():
            score = sum(abs(row[col] - ref_row[col]) for col in a_cols)
            scores[ref_row['pose_name']] = score
    
        closest_match_spec = min(scores, key=scores.get)
        closest_score = scores[closest_match_spec]
    
        if closest_score > 200:
            return 'unknown'
        else:
            suffixes_to_remove = r'(-rgt|-lft|-inv|-rgt-inv|-lft-inv|-center|)$'
            closest_match = re.sub(suffixes_to_remove, '', closest_match_spec)
            return closest_match

In [24]:
data = pd.read_csv('data/ground_truth/processed/upright/pose_data.csv')
ref_legs = pd.read_csv('data/external/positions/legs/pose_data.csv')
ref_grip = pd.read_csv('data/external/positions/grip/pose_data.csv')

results = PoseSpatialClassifier(data, ref_legs, ref_grip)
results.data[['image_filename','secs','frame_no','pos_face','pos_body','pos_legs','pos_grip']]

,image_filename,secs,frame_no,pos_face,pos_body,pos_legs,pos_grip
0,0000_00000000.png,0,0,upright,front,grand-battement-backward,one-hand-basic-grip
1,0000_00000011.png,0,11,upright,back,step-around,one-hand-bridge-arch
2,0000_00000022.png,0,22,upright,back,step-around,baseball-grip
3,0000_00000033.png,0,33,upright,back,passe,one-hand-strong-hold
4,0000_00000044.png,0,44,upright,back,pencil,two-hand-double-bridge-arch
5,0000_00000055.png,0,55,upright,back,fang,armpit-grip
6,0001_00000066.png,1,66,upright,back,releve,two-hand-double-bridge-arch
7,0001_00000077.png,1,77,upright,back,pencil,armpit-grip
8,0001_00000088.png,1,88,upright,front,rolled-toes-releve,armpit-grip
9,0001_00000099.png,1,99,upright,front,step-around,one-hand-cup-grip
